<font color='blue'>

# Goal 

Retrieve the sequence of all proteins encoded in the human genome. 
</font>

In [1]:
import numpy as np
import pandas as pd
import csv

In [2]:
import Bio
from Bio import SeqIO

In [3]:
from Bio.Seq import Seq

<font color='blue'>
This is the human genome annotation https://genome.ucsc.edu/cgi-bin/hgTables
</font>

In [4]:
#HGA = pd.read_csv('human_genome_annotation.txt', sep='\t')
HGA = pd.read_csv('human_genome_annotation.txt', sep='\t')

# Data Preprocessing

This is where I tested how to process the data. I trimmed it to include only the necessary columns and information I would need. I applied this to the first 20 rows to make sure it would work. I then applied it to whole data set and wrote the data out to the file called `dnaData.csv` 

In [5]:
HGA

,#bin,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,score,name2,cdsStartStat,cdsEndStat,exonFrames
0,0,NM_001276352.2,chr1,-,67092164,67134970,67093579,67127240,9,"67092164,67096251,67103237,67111576,67115351,6...","67093604,67096321,67103382,67111644,67115464,6...",0,C1orf141,cmpl,cmpl,"2,1,0,1,2,0,0,-1,-1,"
1,0,NM_001276351.2,chr1,-,67092164,67134970,67093004,67127240,8,"67092164,67095234,67096251,67115351,67125751,6...","67093604,67095421,67096321,67115464,67125909,6...",0,C1orf141,cmpl,cmpl,"0,2,1,2,0,0,-1,-1,"
2,0,NR_075077.2,chr1,-,67092164,67134970,67134970,67134970,10,"67092164,67096251,67103237,67111576,67113613,6...","67093604,67096321,67103382,67111644,67113756,6...",0,C1orf141,none,none,"-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,"
3,0,XM_011541469.1,chr1,-,67092175,67109072,67093004,67103382,5,"67092175,67095234,67096251,67103237,67109028,","67093604,67095421,67096321,67103382,67109072,",0,C1orf141,cmpl,cmpl,"0,2,1,0,-1,"
4,0,XM_011541467.1,chr1,-,67092175,67131183,67093004,67127240,9,"67092175,67095234,67096251,67103237,67111576,6...","67093604,67095421,67096321,67103343,67111644,6...",0,C1orf141,cmpl,cmpl,"0,2,1,0,1,2,0,0,-1,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166918,586,XM_024452505.1,chr22_KI270734v1_random,-,138085,153834,138479,150383,13,"138085,138742,142193,143613,144748,145003,1466...","138667,138831,142292,143789,144895,145096,1467...",0,LOC102724788,cmpl,cmpl,"1,2,2,0,0,0,2,0,0,1,0,-1,-1,"
166919,586,XM_017030169.1,chr22_KI270734v1_random,-,138085,153836,138479,152899,12,"138085,138742,142193,143613,144748,145003,1466...","138667,138831,142292,143789,144895,145096,1467...",0,LOC102724788,cmpl,cmpl,"1,2,2,0,0,0,2,0,0,0,1,0,"
166920,586,XM_017030167.1,chr22_KI270734v1_random,-,138085,153840,138479,152899,13,"138085,138742,142193,143613,144748,145003,1466...","138667,138831,142292,143789,144895,145096,1467...",0,LOC102724788,cmpl,cmpl,"1,2,2,0,0,0,2,0,0,0,1,1,0,"
166921,586,XM_006724936.3,chr22_KI270734v1_random,-,138085,161594,138479,150995,13,"138085,138742,142193,143613,144748,145003,1466...","138667,138831,142292,143789,144895,145096,1467...",0,LOC102724788,cmpl,cmpl,"1,2,2,0,0,0,2,0,0,0,0,-1,-1,"


In [6]:
HGA = HGA.head(20)

In [7]:
HGA = HGA.drop(['#bin', 'txStart', 'txEnd', 'exonCount', 'score', 'cdsStartStat', 'cdsEndStat'], axis = 1)

In [8]:
alist = []
blist = []
elist = []

for idx, row in HGA.iterrows(): 
    
    a = row.exonStarts 
    a = a.split(',')
    a = a[:len(a)-1]
    a = list(map (int,a))
    
    b = row.exonEnds 
    b = b.split(',')
    b = b[:len(b)-1]
    b = list(map (int,b))
    
    e = row.exonFrames 
    e = e.split(',')
    e = e[:len(e)-1]
    e = list(map (int,e))
    
    
    alist.append(a)
    blist.append(b)
    elist.append(e)
    
df = pd.DataFrame({'starts':alist, 'ends':blist, 'frames':elist})
    

In [9]:
HGA = HGA.drop(['exonStarts', 'exonEnds', 'exonFrames'], axis = 1)

In [10]:
merged = pd.merge(left=df, left_index=True, right=HGA, right_index=True, how='inner')

In [11]:
merged

,starts,ends,frames,name,chrom,strand,cdsStart,cdsEnd,name2
0,"[67092164, 67096251, 67103237, 67111576, 67115...","[67093604, 67096321, 67103382, 67111644, 67115...","[2, 1, 0, 1, 2, 0, 0, -1, -1]",NM_001276352.2,chr1,-,67093579,67127240,C1orf141
1,"[67092164, 67095234, 67096251, 67115351, 67125...","[67093604, 67095421, 67096321, 67115464, 67125...","[0, 2, 1, 2, 0, 0, -1, -1]",NM_001276351.2,chr1,-,67093004,67127240,C1orf141
2,"[67092164, 67096251, 67103237, 67111576, 67113...","[67093604, 67096321, 67103382, 67111644, 67113...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]",NR_075077.2,chr1,-,67134970,67134970,C1orf141
3,"[67092175, 67095234, 67096251, 67103237, 67109...","[67093604, 67095421, 67096321, 67103382, 67109...","[0, 2, 1, 0, -1]",XM_011541469.1,chr1,-,67093004,67103382,C1orf141
4,"[67092175, 67095234, 67096251, 67103237, 67111...","[67093604, 67095421, 67096321, 67103343, 67111...","[0, 2, 1, 0, 1, 2, 0, 0, -1]",XM_011541467.1,chr1,-,67093004,67127240,C1orf141
5,"[67092175, 67095234, 67096251, 67103237, 67111...","[67093604, 67095421, 67096321, 67103382, 67111...","[0, 2, 1, 0, 1, 2, 0, 0, -1]",XM_017001276.1,chr1,-,67093004,67127240,C1orf141
6,"[67092175, 67095234, 67096251, 67103237, 67111...","[67093604, 67095421, 67096321, 67103382, 67111...","[0, 2, 1, 0, 1, 2, 0, 0, -1]",XM_011541465.2,chr1,-,67093004,67127240,C1orf141
7,"[67092175, 67095234, 67096251, 67103237, 67111...","[67093604, 67095421, 67096321, 67103382, 67111...","[0, 2, 1, 0, 1, 2, 0, 0, -1]",XM_011541466.2,chr1,-,67093004,67127240,C1orf141
8,"[67093484, 67095353, 67096251, 67103237, 67111...","[67093604, 67095421, 67096321, 67103382, 67111...","[1, 2, 1, 0, 1, 2, 0, 0, -1]",XM_017001277.1,chr1,-,67093569,67127240,C1orf141
9,"[67093484, 67096251, 67115351, 67125751, 67127...","[67093604, 67096321, 67115464, 67125909, 67127...","[2, 1, 2, 0, 0, -1]",XM_011541473.2,chr1,-,67093579,67127240,C1orf141


# Import Trimmed Data

Here I imported the trimmed data from the file dnaData.csv. I applied various functions to the data frame to ensure the data was in the correct format for processing. 

In [12]:
dnaData = pd.read_csv('dnaData.csv')

In [13]:
final_df = merged.merge(dnaData, left_on='chrom', right_on='Name')

In [14]:
final_df = final_df.dropna()

In [15]:
final_df = final_df.apply(lambda x: x.explode() if x.name in ['starts', 'ends','frames'] else x)

In [16]:
final_df = final_df[final_df.frames != -1]

In [17]:
final_df = final_df.rename({"name": "name_to_merge"}, axis=1)

# Processing the Data

Here I processed the data. That is, I used information such as start codon and end codon to retrieve the sequence of protiens from the original DNA sequence. 

In [18]:
alist = []
nlist = []
basesData = pd.DataFrame()

for idx,row in final_df.iterrows(): 
    st = row.starts
    en = row.ends
    cdStart = row.cdsStart
    cdEnd = row.cdsEnd
    nlist.append(row.name_to_merge)
    
    if (st < cdStart):
        st = cdStart 
    if(en > cdEnd):
        en = cdEnd
        
    actual = row.Bases[st:en]
    alist.append(actual)
    
basesData['Name'] = nlist
basesData['Bases_2'] = alist

In [19]:
basesData = basesData.groupby(['Name'])['Bases_2'].apply(lambda x: ''.join(x)).reset_index()

In [20]:
final_df_2 = merged.merge(basesData, left_on='name', right_on='Name')

In [21]:
final_df_2 = final_df_2.dropna()

In [22]:
final_df_2 = final_df_2.drop(['starts', 'ends', 'frames', 'cdsStart', 'cdsEnd'], axis = 1)

In [23]:
final_df_2

,name,chrom,strand,name2,Name,Bases_2
0,NM_001276352.2,chr1,-,C1orf141,NM_001276352.2,TTATGGGATTTGTGTCCTTTTGTTCCTTTTATTTCTATCACCTTCT...
1,NM_001276351.2,chr1,-,C1orf141,NM_001276351.2,TTATGAGGCATTTAAAATTTCATTTGATAAGTTTAACAAATTATCC...
2,XM_011541469.1,chr1,-,C1orf141,XM_011541469.1,TTATGAGGCATTTAAAATTTCATTTGATAAGTTTAACAAATTATCC...
3,XM_011541467.1,chr1,-,C1orf141,XM_011541467.1,TTATGAGGCATTTAAAATTTCATTTGATAAGTTTAACAAATTATCC...
4,XM_017001276.1,chr1,-,C1orf141,XM_017001276.1,TTATGAGGCATTTAAAATTTCATTTGATAAGTTTAACAAATTATCC...
5,XM_011541465.2,chr1,-,C1orf141,XM_011541465.2,TTATGAGGCATTTAAAATTTCATTTGATAAGTTTAACAAATTATCC...
6,XM_011541466.2,chr1,-,C1orf141,XM_011541466.2,TTATGAGGCATTTAAAATTTCATTTGATAAGTTTAACAAATTATCC...
7,XM_017001277.1,chr1,-,C1orf141,XM_017001277.1,TCATGGAAAATTATGGGATTTGTGTCCTTTTGTTCTTAATTGATAA...
8,XM_011541473.2,chr1,-,C1orf141,XM_011541473.2,TTATGGGATTTGTGTCCTTTTGTTCCTTTTATTTCTATCACCTTCT...
9,XM_011541472.1,chr1,-,C1orf141,XM_011541472.1,CTAAGAGACCAACAGAATCCAATGGTTTCCTAGGTTTTTTTTCAAT...


# Writing to a file 

Here, I wrote the processed data to a file. I used the file for homework submission. 

In [24]:
f=open('foo.fa', "w")

for idx,row in final_df_2.iterrows(): 
    strand = row.strand
    bases = row.Bases_2
    bases = Seq(bases)
    name1 = row.Name
    name2 = row.name2
    
    if (strand == '-'):
        bases = bases.reverse_complement()
            
    my_seq = bases.translate(to_stop=True)
    fasta_format_string = ">%s:%s" % (name1, name2)
    print(fasta_format_string, file=f)
    print(fasta_format_string)
    
    fasta_format_string = "%s" % my_seq
    print(fasta_format_string, file=f)
    print(fasta_format_string)
    
f.close()    

>NM_001276352.2:C1orf141
MAEKILEKLDVLDKQAEIILARRTKINRLQSEGRKTTMAIPLTFDFQLEFEEALATSASKAISKIKEDKSCSITKSKMHVSFKCEPEPRKSNFEKSNLRPFFIQTNVKNKESESTEPVEEHLKSRSIRPYLYLKDTTEMENAGPLNVLYSQHRQACRRSLGSTDFSPMFNIQSNAHKKEKDSTLFTAQIEKKPRKPLDSVGLLEGDRNKRNKRTQIP
>NM_001276351.2:C1orf141
MAEKILEKLDVLDKQAEIILARRTKINRLQSEGRKTTMAIPLTFDFQLEFEEALATSASKAISKIKEDKSCSITKSKMHVSFKCEPEPRKSNFEKSNLRPFFIQTNVKNKESESTAQIEKKPRKPLDSVGLLEGDRNKRKKSPQMNDFNIKENKSVRNYQLSKYRSVRKKSLLPLCFEDELKNPHAKIVNVSPTKTVTSHMEQKDTNPIIFHDTEYVRMLLLTKNRFSSHPLENENIYPHKRTNFILERNCEILKSIIGNQSISLFKPQKTMPTVQRKDIQIPMSFKAGHTTVDDKLKKKTNKQTLENRSWNTLYNFSQNFSSLTKQFVGYLDKAVIHEMSAQTGKFERMFSAGKPTSIPTSSALPVKCYSKPFKYIYELNNVTPLDNLLNLSNEILNAS
>XM_011541469.1:C1orf141
MENAGPLNVLYSQHRQACRRSLGSTDFSPMFNIQSNAHKKEKDSTLFTAQIEKKPRKPLDSVGLLEGDRNKRKKSPQMNDFNIKENKSVRNYQLSKYRSVRKKSLLPLCFEDELKNPHAKIVNVSPTKTVTSHMEQKDTNPIIFHDTEYVRMLLLTKNRFSSHPLENENIYPHKRTNFILERNCEILKSIIGNQSISLFKPQKTMPTVQRKDIQIPMSFKAGHTTVDDKLKKKTNKQTLENRSWNTLYNFSQNFSSLTKQFVGYLDKAVIHEMSAQTGKFERMFSAGKPTSIPTSSALPVKCYSKP